In [2]:
pip install pinecone -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pinecone-text -q

## importance of hybrid search is that it take into consideration of both

1. semantic search ( making use of cosine similarity and vector db is highly dense )
2. syntatic search ( making sparse database by making use of BOW, One Hot Encoding,TF-IDF )


In [14]:
from pinecone import Pinecone, ServerlessSpec
import json

## retrieve api key
with open("../key.json") as f:
    keys = json.load(f)
    pine_api=keys["pine_cone_api"]
    
pc = Pinecone(api_key=pine_api)

In [15]:
if pc:
    print("Pinecone connected")
else:
    print("Pinecone not connected")

Pinecone connected


In [17]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

## creating index if it does not exist
if "hybriddb" not in pc.list_indexes():
    pc.create_index(name="hybriddb", dimension=384, metric="dotproduct",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
    )



In [49]:
index=pc.Index("hybriddb")

In [ ]:
## vector embedding and sparse metrics

import os
from langchain_huggingface import HuggingFaceEmbeddings

embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

len(embedding.embed_query("hello"))

### the len of embedding must match the dimension of the index or else it will throw an error
### while embedding into database and hence always check the dimension of embedding in the hugging face model

384

In [26]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt_tab to /home/vscode/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [28]:
sentences=[
    "In 2025 i will visit india",
    "In 2024 i visited ireland",
    "In 2023 i visited germany",]

bm25_encoder.fit(sentences)

100%|██████████| 3/3 [00:00<00:00, 99.05it/s]


In [44]:
### store the value of the encoder in a file into json format

bm25_encoder.dump("bm25_encoder.json")

## loading the encoder
bn25_encoder = BM25Encoder().load("bm25_encoder.json")

In [50]:
retriever = PineconeHybridSearchRetriever(
    embeddings=embedding,
    sparse_encoder=bn25_encoder,
    index=index
)

In [51]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0xffff3cdc94e0>, index=<pinecone.db_data.index.Index object at 0xffff151b5cf0>)

In [52]:
sentences

['In 2025 i will visit india',
 'In 2024 i visited ireland',
 'In 2023 i visited germany']

In [53]:
retriever.add_texts(
    ["In 2025 i will visit india",
    "In 2024 i visited ireland",
    "In 2023 i visited germany"])

100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


In [56]:
retriever.invoke("In which country i visited last year")

[Document(metadata={'score': 0.326980829}, page_content='In 2023 i visited germany'),
 Document(metadata={'score': 0.303880274}, page_content='In 2024 i visited ireland'),
 Document(metadata={'score': 0.233867079}, page_content='In 2025 i will visit india')]